In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Preprocess data

In [2]:
# Read, then decode for py2 compat.
text = open("data/tale of kieu.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 110585 characters


In [3]:
print(text[:200])

1.Trăm năm trong cõi người ta,
Chữ tài chữ mệnh khéo là ghét nhau.
Trải qua một cuộc bể dâu,
Những điều trông thấy mà đau đớn lòng.
5.Lạ gì bỉ sắc tư phong,
Trời xanh quen thói má hồng đánh ghen.


In [4]:
text = re.sub(r'[0-9]+.', '', text.lower())
text = re.sub(r'[\.,!?;()-]', '', text.lower())

In [5]:
print(text[:250])

trăm năm trong cõi người ta
chữ tài chữ mệnh khéo là ghét nhau
trải qua một cuộc bể dâu
những điều trông thấy mà đau đớn lòng
lạ gì bỉ sắc tư phong
trời xanh quen thói má hồng đánh ghen
cảo thơm lần giở trước đèn
phong tình cổ lục còn truyền s


In [6]:
# get unique characters from origin text
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

92 unique characters


In [7]:
sample_text = ['I am batman', 'i am ironman']
chars = tf.strings.unicode_split(sample_text, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'I', b' ', b'a', b'm', b' ', b'b', b'a', b't', b'm', b'a', b'n'],
 [b'i', b' ', b'a', b'm', b' ', b'i', b'r', b'o', b'n', b'm', b'a', b'n']]>

In [8]:
# convert characters to numeric id
idxs_from_chars = tf.keras.layers.StringLookup(
    vocabulary = list(vocab), mask_token=None
)

In [9]:
idxs = idxs_from_chars(chars)
idxs

<tf.RaggedTensor [[0, 3, 5, 15, 3, 6, 5, 22, 15, 5, 16],
 [12, 3, 5, 15, 3, 12, 20, 17, 16, 15, 5, 16]]>

In [10]:
chars_from_idxs = tf.keras.layers.StringLookup(
    vocabulary = idxs_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [11]:
chars = chars_from_idxs(idxs)
chars

<tf.RaggedTensor [[b'[UNK]', b' ', b'a', b'm', b' ', b'b', b'a', b't', b'm', b'a', b'n'],
 [b'i', b' ', b'a', b'm', b' ', b'i', b'r', b'o', b'n', b'm', b'a', b'n']]>

In [12]:
def text_from_idxs(idxs):
    return tf.strings.reduce_join(chars_from_idxs(idxs), axis=-1)

In [13]:
sample_text = text_from_idxs(idxs)
sample_text

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'[UNK] am batman', b'i am ironman'], dtype=object)>

In [14]:
all_idxs = idxs_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))
all_idxs

<tf.Tensor: shape=(104095,), dtype=int64, numpy=array([22, 20, 43, ...,  5, 16, 11])>

In [15]:
idxs_dataset = tf.data.Dataset.from_tensor_slices(all_idxs)

In [16]:
SEQ_LENGTH = 100

In [17]:
sequences = idxs_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

In [18]:
for seq in sequences.take(5):
    print(text_from_idxs(seq).numpy().decode('utf-8'))
    print("===")

trăm năm trong cõi người ta
chữ tài chữ mệnh khéo là ghét nhau
trải qua một cuộc bể dâu
những điều
===
 trông thấy mà đau đớn lòng
lạ gì bỉ sắc tư phong
trời xanh quen thói má hồng đánh ghen
cảo thơm l
===
ần giở trước đèn
phong tình cổ lục còn truyền sử xanh
rằng năm gia tĩnh triều minh
bốn phương phẳn
===
g lặng hai kinh vững vàng
có nhà viên ngoại họ vương
gia tư nghĩ cũng thường thường bực trung
một 
===
trai con thứ rốt lòng
vương quan là chữ nối dòng nho gia
đầu lòng hai ả tố nga
thúy kiều là chị em
===


In [19]:
def split_input_target(sequence):
    input = sequence[:-1]
    target = sequence[1:]
    return input, target

In [20]:
split_input_target('i am batman')

('i am batma', ' am batman')

In [21]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_idxs(input_example).numpy().decode('utf-8'))
    print("Target:", text_from_idxs(target_example).numpy().decode('utf-8'))

Input : trăm năm trong cõi người ta
chữ tài chữ mệnh khéo là ghét nhau
trải qua một cuộc bể dâu
những điề
Target: răm năm trong cõi người ta
chữ tài chữ mệnh khéo là ghét nhau
trải qua một cuộc bể dâu
những điều


In [22]:
# Batch size
BATCH_SIZE = 64

VOCAB_SIZE = len(idxs_from_chars.get_vocabulary())  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Build model

In [23]:
# Build model
class KieuModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_0 = tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense_0 = tf.keras.layers.Dense(1024)
        self.dropout_0 = tf.keras.layers.Dropout(0.3)
        self.dense_1 = tf.keras.layers.Dense(512)
        self.dropout_1 = tf.keras.layers.Dropout(0.3)
        self.dense_2 = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        # LSTM 1
        if states == None:
            state_h, state_c = self.lstm_0.get_initial_state(x)
            states = [state_h, state_c]
        x, state_h, state_c = self.lstm_0(x, initial_state=states, training=training)
        # FC
        x = self.dense_0(x, training=training)
        x = self.dropout_0(x)
        x = self.dense_1(x, training=training)
        x = self.dropout_1(x)
        x = self.dense_2(x, training=training)

        if return_state:
            return x, [state_h, state_c]
        else:
            return x

In [24]:
model = KieuModel(
    vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, rnn_units=RNN_UNITS
)

In [25]:
for input_example, target_example in dataset.take(1):
    example_prediction = model(input_example)
    print(input_example)

tf.Tensor(
[[30  3 22 ... 64 22  3]
 [11 48 47 ... 16 11  2]
 [12  3 14 ...  1 51 26]
 ...
 [10  3 22 ...  1 16 27]
 [10 49 12 ... 21 52 23]
 [ 3 22 79 ...  7 11 58]], shape=(64, 100), dtype=int64)


In [26]:
model.summary()

Model: "kieu_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23808     
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 dense (Dense)               multiple                  1049600   
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  524800    
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  4

In [27]:
# To get actual predictions from the model you need to sample from the output distribution
sampled_indices = tf.random.categorical(example_prediction[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 0, 11, 33, 29, 42, 46, 82, 20, 41, 83, 61, 74, 10,  6, 60, 73, 35,
       16, 44, 58, 59, 92, 65, 31, 56, 24, 16, 34, 80, 40, 29, 29, 46,  9,
       77, 19, 21, 16, 80,  9, 11, 89, 90, 73, 58, 92, 15, 60,  8, 39, 10,
       26, 62, 88, 49, 18, 83, 80, 87, 32, 60,  3, 91, 30, 82, 69, 50, 22,
       46, 25,  7, 72, 56, 91, 20, 14,  1, 16, 21, 34, 67, 65, 16, 50, 50,
       79, 84, 74, 50, 12, 51, 55, 53, 66, 75, 24, 26, 82,  3, 17])

In [28]:
print("Input:\n", text_from_idxs(input_example[0]).numpy().decode('utf-8'))
print()
print("Next Char Predictions:\n", text_from_idxs(sampled_indices).numpy().decode('utf-8'))

Input:
 ã thẹn nàng oanh
lại thua ả lý bán mình hay sao
cỗi xuân tuổi hạc càng cao
một cây gánh vác biết 

Next Char Predictions:
 [UNK]hêâýũợrúụẹồgbặốínđẳẵỹềèắvnìởùââũeộqsnởehựỳốẳỹmặdõgyẻữạpụởửéặ ỷãợỉảtũxcỏắỷrl
nsìễềnảảờủồảiấậẩểổvyợ o


In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=loss)

In [31]:
def scheduler(epoch, lr):
    if epoch > 1000 and (epoch % 100) == 0:
        return lr * 0.5
    else:
        return lr

In [32]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [33]:
history = model.fit(dataset, epochs=2000, verbose=2)

Epoch 1/2000
16/16 - 6s - loss: 4.3216 - 6s/epoch - 385ms/step
Epoch 2/2000
16/16 - 1s - loss: 3.7216 - 1s/epoch - 74ms/step
Epoch 3/2000
16/16 - 1s - loss: 3.5616 - 1s/epoch - 65ms/step
Epoch 4/2000
16/16 - 1s - loss: 3.5004 - 1s/epoch - 65ms/step
Epoch 5/2000
16/16 - 1s - loss: 3.4593 - 1s/epoch - 65ms/step
Epoch 6/2000
16/16 - 1s - loss: 3.4140 - 1s/epoch - 65ms/step
Epoch 7/2000
16/16 - 1s - loss: 3.3406 - 1s/epoch - 66ms/step
Epoch 8/2000
16/16 - 1s - loss: 3.2289 - 1s/epoch - 66ms/step
Epoch 9/2000
16/16 - 1s - loss: 3.0924 - 1s/epoch - 73ms/step
Epoch 10/2000
16/16 - 1s - loss: 2.9518 - 1s/epoch - 66ms/step
Epoch 11/2000
16/16 - 1s - loss: 2.8156 - 1s/epoch - 65ms/step
Epoch 12/2000
16/16 - 1s - loss: 2.7232 - 1s/epoch - 65ms/step
Epoch 13/2000
16/16 - 1s - loss: 2.6562 - 1s/epoch - 64ms/step
Epoch 14/2000
16/16 - 1s - loss: 2.5911 - 1s/epoch - 65ms/step
Epoch 15/2000
16/16 - 1s - loss: 2.5469 - 1s/epoch - 65ms/step
Epoch 16/2000
16/16 - 1s - loss: 2.4761 - 1s/epoch - 65ms/step


In [34]:
model.save_weights('my_model/kieu_gpt')

In [35]:
import json
idxs_from_chars_cfg = idxs_from_chars.get_config()
with open("idxs_lookup_config.json", "w") as outfile:
    json.dump(idxs_from_chars_cfg, outfile)
    
# tf.keras.layers.StringLookup(**idxs_from_chars_cfg)

# Generate text

In [36]:
class PoemTeller(tf.keras.Model):
    def __init__(self, model, chars_from_idxs, idxs_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_idxs = chars_from_idxs
        self.idxs_from_chars = idxs_from_chars

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.idxs_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                            return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_idxs(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [37]:
teller = PoemTeller(model, chars_from_idxs, idxs_from_chars, 0.1)

In [38]:
inp = "em"

lines = 10
start = time.time()
states = None
next_char = tf.constant([inp])
result = [next_char]

i=0
while i < lines:
    next_char, states = teller.generate_one_step(next_char, states=states)
    result.append(next_char)
    if next_char[0].numpy().decode('utf-8') == '\n':
        i += 1

result = tf.strings.join(result)
output_text = result[0].numpy().decode('utf-8')
end = time.time()
print(output_text, '\n\n' + '-'*80)
print('Run time:', end - start)

em
tưng bừng sắm sửa áo xiêm
biện dâng một lễ xa đem tấc thành
nhà lan thanh vắng một mình
ngẫm chần đồ đến ai lầu chương trời
nôi then những thịnh trăng vài
trời đâu bắng động tiện gười nhun dan
xóc ngu tà mệt ai
trong nhần đã thủ phận hồi cho ang
đưa trầy chịp dau tơ đàn
 

--------------------------------------------------------------------------------
Run time: 1.9405808448791504
